<a href="https://colab.research.google.com/github/Sayandip2023/Fashion-MNIST-Classification/blob/main/KAN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np



In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images to the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Split the training data into training and validation sets
x_train, x_val = x_train[:40000], x_train[40000:]
y_train, y_val = y_train[:40000], y_train[40000:]


In [ ]:
def create_hybrid_kan_cnn(input_shape):
    # Define the CNN for feature extraction
    cnn_input = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(cnn_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.Flatten()(x)
    cnn_output = layers.Dense(128, activation='relu')(x)

    # Define the KAN for function approximation
    kan_input = layers.Input(shape=(128,))
    y = layers.Dense(256, activation='relu')(kan_input)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.Dense(64, activation='relu')(y)
    kan_output = layers.Dense(10, activation='softmax')(y)  # Assuming 10 classes for classification

    # Combine CNN and KAN
    combined_input = cnn_output
    kan = models.Model(inputs=kan_input, outputs=kan_output)
    combined_output = kan(combined_input)

    # Full model
    model = models.Model(inputs=cnn_input, outputs=combined_output)

    return model

In [ ]:
# Define input shape based on your image data
input_shape = (32, 32, 3)  # CIFAR-10 images are 32x32 RGB images

# Create the hybrid model
model = create_hybrid_kan_cnn(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")